In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\hi\appdata\local\temp\pip-req-build-xky7gb9j


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [3]:
!pip install qkeras


  Obtaining dependency information for qkeras from https://files.pythonhosted.org/packages/52/39/126bae2890a506b2e70d71af3e1b06ba3f3109a68182999531028a0a01c7/QKeras-0.9.0-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for tensorflow-model-optimization>=0.2.1 from https://files.pythonhosted.org/packages/5a/a0/f38c9ce977285d3f347f6d64cf227924fe9fc97d780d6df4a55563e311af/tensorflow_model_optimization-0.8.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for keras-tuner>=1.0.1 from https://files.pythonhosted.org/packages/db/5d/945296512980b0827e93418514c8be9236baa6f0a1e8ca8be3a2026665b0/keras_tuner-1.4.7-py3-none-any.whl.metadata
  Obtaining dependency information for kt-legacy from https://files.pythonhosted.org/packages/16/53/aca9f36da2516db008017db85a1f3cafaee0efc5fc7a25d94c909651792f/kt_legacy-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency informa

In [4]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from qkeras.qlayers import QRNN

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Define the QRNN model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(QRNN(128, window_size=3, return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=50)

# Save the trained model
model.save('lung_detection_qrnn.h5')


ImportError: cannot import name 'QRNN' from 'qkeras.qlayers' (C:\Users\Hi\anaconda3\Lib\site-packages\qkeras\qlayers.py)